In [6]:
import pandas as pd
from pyvi import ViTokenizer, ViPosTagger
import nltk
import numpy as np

In [60]:
# filename = "./dataset/bạn bè.txt"
# df = pd.read_fwf(filename,header = None, encoding="utf-8")
filename =["bạn bè.txt","các câu hỏi phức tạp.txt","du lịch.txt","gia đình.txt","giải trí.txt",
           "học tập.txt","nghề nghiệp.txt","nghỉ lễ.txt","người yêu.txt","robot.txt","shoping.txt",
           "sở thích.txt","tdtu.txt","thông tin cá nhân.txt","trò chuyện về đi ăn.txt","tán gẫu.txt","đất nước.txt","địa chỉ.txt"]
# df = pd.read_fwf(filename,header = None)
# df.head()
print(len(filename))

18


In [61]:
df = df[0]

In [62]:
temp_ques = []
temp_ans = []
tag = []
for k in range(len(filename)):
    with open('dataset/' + filename[k], encoding="utf-8") as f:
        lines = f.readlines()
    for i in range(len(lines)):
        if lines[i].startswith('__eou__'):
            continue
        else:
            part = lines[i].strip('__eou__').split('__eou__')
            temp_ques.append(ViTokenizer.tokenize(part[0].lower().strip()))
            temp_ans.append(ViTokenizer.tokenize(part[1].lower().strip()))
            tag.append(filename[k].split(".")[0])

In [75]:
data = pd.DataFrame({'Question':temp_ques,'Answer':temp_ans,'Tag':tag})
data.head()

,Question,Answer,Tag
0,thích đánh_lộn không ?,ngon nhà_vô,bạn bè
1,solo yasua không,chấp lun 2 mạng đầu,bạn bè
2,mai đi picnic không ?,mai bận học rồi,bạn bè
3,mai học ca mấy vậy ?,mai học ca 3,bạn bè
4,còn tiền không ?,còn chết liền,bạn bè


In [64]:
ques = np.array(data["Question"])
ans = np.array(data["Answer"])
ques_sentences=[nltk.word_tokenize(sent) for sent in ques]
ans_sentences=[nltk.word_tokenize(sent) for sent in ans] 

In [65]:
def clean_sentences(sentences):
    Punc = {'.', ',', '...', '-', '“', '”', ':', '(', ')', '"', '!', '&', ';', '?', '*', ']', '>', '…', '’',"``","''"}
    clean_sent = []
    for sent in sentences:
        temp =[]
        for item in sent:
            if item in Punc:
                continue
            elif(item.isdigit()):
                temp.append("Num")
            else:
                temp.append(item)
        clean_sent.append(temp)
    return clean_sent

In [66]:
def clean_one_sentence(sentences):
    Punc = {'.', ',', '...', '-', '“', '”', ':', '(', ')', '"', '!', '&', ';', '?', '*', ']', '>', '…', '’',"``","''"}
    clean_sent = []
    for sent in sentences:
        if sent in Punc:
            continue
        elif(sent.isdigit()):
            clean_sent.append("Num")
        else:
            clean_sent.append(sent)
    return clean_sent

In [67]:
clean_ques = clean_sentences(ques_sentences)
clean_ans = clean_sentences(ans_sentences)
# print(len(clean_ques))
# print(len(clean_ans))

### Word2Vec

In [68]:
from gensim.models import Word2Vec
model = Word2Vec(clean_ques, min_count = 1,size = 100)

In [69]:
def convert2vec(words):
    _sum = np.array([0]*100)
    for word in words:
        if not(word in model.wv.vocab):
            continue
        vec = model.wv[word]
        _sum = _sum + vec
    return _sum/len(words)

In [121]:
X = []
for sent in clean_ques:
    if len(sent) == 0:
        X.append(np.array([0]*100))
    else:
        X.append(convert2vec(sent))
X = np.array(X)

## Text classification

In [93]:
text_classify_data = data

In [94]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoded_label = encoder.fit_transform(text_classify_data.Tag)
text_classify_data.Tag = encoded_label

In [95]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test =  train_test_split(X, data.Tag, test_size=0.2, random_state=0)

In [96]:
from sklearn.tree import DecisionTreeClassifier

clf_decision_word2vec = DecisionTreeClassifier()
clf_decision_word2vec.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [98]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

test_predictions_word2vec = clf_decision_word2vec.predict(X_test)
print("Accuracy score: " + str(accuracy_score(y_test, test_predictions_word2vec)))
print(classification_report(y_test, test_predictions_word2vec))

Accuracy score: 0.3842869342442357
              precision    recall  f1-score   support

           0       0.33      0.35      0.34       124
           1       0.17      0.38      0.23        16
           2       0.40      0.36      0.38        47
           3       0.28      0.44      0.34        75
           4       0.45      0.45      0.45        51
           5       0.45      0.48      0.46        48
           6       0.39      0.35      0.37        86
           7       0.22      0.29      0.25        28
           8       0.46      0.41      0.43        96
           9       0.12      0.07      0.09        40
          10       0.33      0.38      0.36        73
          11       0.53      0.50      0.51        52
          12       0.26      0.15      0.19        61
          13       0.60      0.63      0.61       151
          14       0.31      0.20      0.24        55
          15       0.39      0.33      0.36        45
          16       0.28      0.26      0.27   

In [99]:
from sklearn.ensemble import RandomForestClassifier

RFC_model = RandomForestClassifier(n_estimators=1000, random_state=0)
RFC_model.fit(X_train, y_train) 

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [100]:
y_pred = RFC_model.predict(X_test)

In [101]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.34      0.60      0.43       124
           1       0.70      0.44      0.54        16
           2       0.46      0.40      0.43        47
           3       0.38      0.47      0.42        75
           4       0.59      0.53      0.56        51
           5       0.60      0.50      0.55        48
           6       0.63      0.44      0.52        86
           7       0.62      0.36      0.45        28
           8       0.55      0.44      0.49        96
           9       0.27      0.10      0.15        40
          10       0.33      0.67      0.45        73
          11       0.65      0.60      0.62        52
          12       0.80      0.33      0.47        61
          13       0.62      0.72      0.67       151
          14       0.34      0.20      0.25        55
          15       0.48      0.27      0.34        45
          16       0.44      0.18      0.26        38
          17       0.48    

In [140]:
from scipy import spatial

#Find the nearest vector to input question
def get_answer(input_sent):
    # Clean input_sent
    input_sent = ViTokenizer.tokenize(input_sent).lower().strip()
    input_sent = clean_one_sentence(nltk.word_tokenize(input_sent))
    
    #Turn input sentence into vector
    vector_input = convert2vec(input_sent)
    
    #Predict tag of input sentence
    tag_code = clf_decision_word2vec.predict([vector_input])
    print("Tag code: " + str(tag_code[0]))
    tag_ques = text_classify_data[text_classify_data.Tag == tag_code[0]]
    
    #Find the nearest question in train data | cosine similarity
    cosin_lst = []
    for idx in range(len(tag_ques)):
        cosin_lst.append([1 - spatial.distance.cosine(vector_input, X[idx]),idx])
    possion = max(cosin_lst)[1]
    print(ans[possion])

In [143]:
get_answer("Bạn nữ có hào hứng khi trò chuyện không?")

Tag code: 11
có mình thích nấu_ăn và mình cũng hay nấu_ăn khi rãnh rỗi


## Counter

In [69]:
import math
from collections import Counter

In [70]:
def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

In [134]:
def chat(input_):
    x = ViTokenizer.tokenize(input_).lower().strip()
    text1 = clean_one_sentence(nltk.word_tokenize(x))
    vector1 = Counter(text1)
    cosin = []
    # temp = []

    for i in range(len(clean_ques)):
        text2 = clean_ques[i]
        vector2 = Counter(text2)
        cosin.append([get_cosine(vector1, vector2),i])
    possion = max(cosin)[1]
    print(ans[possion])

In [135]:
chat("Dạo này, Ổn không?>>")

buồn lắm bạn à , crush unfriend trên facebook tui rồi , giờ tôi muốn nhắn_tin với crush cũng k được nữa


In [ ]:
temp = "dữ liệu chatbot - dự án cuối kỳ"